In [1]:
#Coppied and modified from Tamas Nagy code 2021: 
#Evelyn Strickland 2022, for use on my imaging setup, multithreading doesn't work just yet with this????
#V1.0, 2024, included in toolset for analysis pipeline!

from csbdeep.utils import normalize
from tqdm.notebook import tqdm as tqdm
from pathlib import Path
from stardist.models import StarDist2D 
from tifffile import TiffFile, TiffWriter
import tensorflow as tf

In [2]:
#General command to run stardist on a 2 channel tif, or a single channel nuclei only tif: 
def run_stardist(filepath, model):
    
    #Get the filename directory, ome-tag, create saveloc, and virutally open the image: 
    dirname = filepath.parent
    omename = filepath.name.split('.', 1)[0]
    savedir = filepath.parent.parent / 'segmented_nuclei'
    img = TiffFile(filepath)
    
    #Open a writer to output each slice to a new segmeneted tif at saveloc: 
    with TiffWriter(savedir.joinpath(omename + "_segmented.tif")) as tif:
        page = img.pages
        series = img.series[0]
        #Check to see if the images are both a calcium and nuclei channel. If so, we expect the images to have the calcium channel FIRST, nuclei SECOND
        if len(page) == 2 * series.get_shape()[0]:
            pages = page[1::2]
        else:
            pages = page
        #Take the pages, normalize them, then pass them over to the stardist model for segmentation: 
        for page in tqdm(pages):
            pg = normalize(page.asarray(), 1, 99.8, axis=(0,1))
            labels, _ = model.predict_instances(pg)
            tif.write(labels.astype('uint16'), contiguous=True)


In [3]:
#import the model from 20220902 custom trained model (included in archive)
model = StarDist2D(None, name='stardist', basedir='20220902_model')


Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.495435, nms_thresh=0.3.


2023-02-14 08:57:52.517091: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-14 08:57:52.543514: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-14 08:57:52.543642: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-14 08:57:52.544121: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
#SET THIS BEOFORE RUNNING:
folder = '' #Dataset you want to analyze.
folderpath = Path(folder, )
img_folder = folderpath / 'images'
img_paths = list(img_folder.glob('*.tif'))

#Quick dummy check: 
if len(img_paths) == 0:
    print("Check filepath, no images detected...")


In [ ]:
#Main Running Loop: 
for file in img_paths:
    
    dirname = file.parent
    omename = file.name.split('.', 1)[0]
    savedir = file.parent.parent / 'segmented_nuclei'
    
    #If the segmentation file already exists, skip this in the loop
    if savedir.joinpath(omename + "_segmented.tif").exists():
        print('Image {} already has a segmentation!'.format(omename))
        continue
    
    print('Running stardist on {}.tif'.format(file.stem))
    run_stardist(file,model)

Image 0 already has a segmentation!
Image 1 already has a segmentation!
Running stardist on 6.tif


  0%|          | 0/200 [00:00<?, ?it/s]

2023-02-14 08:58:08.184505: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8302
2023-02-14 08:58:09.128553: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Image 3 already has a segmentation!
Image 4 already has a segmentation!
Image 5 already has a segmentation!
Running stardist on 7.tif


  0%|          | 0/200 [00:00<?, ?it/s]

Running stardist on 8.tif


  0%|          | 0/200 [00:00<?, ?it/s]